# Import MODFLOW 6 models from Geomodelr

Welcome to Geomodelr webinar. Here we are going to explain how to load and run our models created and exported from Geomodelr. Furthemore, we will show how to get the results and save them in the _vtk_ file.

For this goal, we need the last version of flopy (3.2.12). Furthemore, we are using the vtk library, version 8.2.0, json library, version 2.0.9 and numpy, version 1.16.4.

In [1]:
import numpy as np
import flopy
import vtk
import json
from vtk.util import numpy_support

print('numpy version {}'.format(np.__version__))
print('flopy version {}'.format(flopy.__version__))
print('json version {}'.format(json.__version__))
print(vtk.vtkVersion.GetVTKSourceVersion())

flopy is installed in D:\Programas_Instalados\Anaconda3\envs\advanced-math\lib\site-packages\flopy
numpy version 1.16.4
flopy version 3.2.12
json version 2.0.9
vtk version 8.2.0


## Load simulation

We define the name of the simulation and its location. On the other hand, we must to set the location of mf6.exe (including the executable)

In [2]:
file_name = 'mf6_webinar' # Name of the study

sim_location = 'D:/CompartidaVB/Modflow/Webinar_2019/modflow6/{}/'.format(file_name)
exe_location = 'C:/WRDAPP/mf6.0.4/mf6.0.4/bin/mf6.exe' # mf6 exucatable location

After that, we load the simulation

In [3]:
sim = flopy.mf6.MFSimulation.load(sim_ws=sim_location , exe_name=exe_location)
model = sim.get_model(file_name)

loading simulation...
  loading simulation name file...
  loading tdis package...
  loading model gwf6...
    loading package disu...
    loading package riv...
    loading package wel...
    loading package npf...
    loading package ic...
    loading package oc...
  loading ims package mf6_webinar...


In [4]:
model_packages = model.package_names #All packages of the model
print('Model packges: {}\n'.format(model_packages))

sim_packages = sim.package_key_dict.keys()
print('Simulation packges: {}\n'.format(sim_packages))


Model packges: ['disu', 'riv_0', 'wel', 'npf', 'ic', 'oc']

Simulation packges: dict_keys(['nam', 'tdis', 'ims'])



### DISU Package
It is the package that contains all the data about mesh.


In [5]:
disu = model.get_package('disu')

print('Cells info:\n\n{}'.format(disu.cell2d.array)) # Cells array

Cells info:

[(     0, 1067330.93, 1073391.66, 7,     7,     6,     5,     4, 3, 2, 1, None, None, None, None)
 (     1, 1067330.93, 1073391.66, 7,     7,     6,     5,     4, 3, 2, 1, None, None, None, None)
 (     2, 1067330.93, 1073391.66, 7,     7,     6,     5,     4, 3, 2, 1, None, None, None, None)
 ...
 (358476, 1066760.63, 1063718.83, 4, 47886, 42517, 42518, 45406, None, None, None, None, None, None, None)
 (358477, 1066760.63, 1063718.83, 4, 47886, 42517, 42518, 45406, None, None, None, None, None, None, None)
 (358478, 1066760.63, 1063718.83, 4, 47886, 42517, 42518, 45406, None, None, None, None, None, None, None)]


In [6]:
print('Vertices info:\n\n{}'.format(disu.vertices.array)) # vertices array

Vertices info:

[(    0, 1067350.66, 1073402.91) (    1, 1067347.78, 1073405.99)
 (    2, 1067333.06, 1073406.41) ... (47883, 1066699.42, 1063707.81)
 (47884, 1066723.9 , 1063707.81) (47885, 1066748.39, 1063707.81)]


In [7]:
print('Conection info:\t{}'.format(disu.ja.array)) # vertices array

Conection info:	[    -1      2     14 ... 341218 358463 358478]


## Riv package
This package contains all corresponding nodes to rivers. These data are organized as (node, stage, conductance, bottom, name).

In [8]:
riv = model.get_package('riv')
print(riv.stress_period_data)

{internal}
([((1396,), 0., 0., 0., 'river_0') ((1410,), 0., 0., 0., 'river_0')
 ((1425,), 0., 0., 0., 'river_0') ... ((10620,), 0., 0., 0., 'river_15')
 ((347908,), 0., 0., 0., 'river_15') ((347923,), 0., 0., 0., 'river_15')])



By default, Geomodelr exports rivers data (stage, conductance and bottom) equal to 0.0 and the names are enumerates from "river_0" to "river_(n-1)", where n is the number of rivers. To edit the rivers data , we have to load rivers_name.json file. This file contains the original name the of the rivers (names in the shapefile) and its correponding enumerated name. We do this to avoid runtime errors because non-ASCII names are not allowed.

In [9]:
json_file = sim_location + 'rivers_name.json'
with open(json_file) as json_file:  
    river_names = json.load(json_file)
print("Rivers names:\n{}".format(river_names))

Rivers names:
{'river_12': 'CD_EL_MOLINO', 'river_13': 'Q._CAIBO', 'river_10': 'CD_MALPASO', 'river_11': 'RIO_ICABUCO', 'river_14': 'Q._BLANCA', 'river_15': 'Q._LOS_CONFINES', 'river_8': 'Q_EL_BARRIAL', 'river_9': 'CD_EL_ROSAL', 'river_0': 'CD_LAS_LARA', 'river_1': 'Q._VOLADOR', 'river_2': 'Q._RUBIO', 'river_3': 'RIO_BOSQUE', 'river_4': 'Q._LA_LAJA', 'river_5': 'Q_TASVITA', 'river_6': 'Q._BOSQUE', 'river_7': 'Q._COLORADA'}


This file is usefull if the stage, conductance and bottom is different for each river or if the user wants to chaged the river names to the original ones (if they are ASCII strings), but in this case, we will define the rivers parameters as follow:
- stage = top - 0.5 (the stage of the river is located 50cm under the topography)
- conductance = 30.
- bottom = top - 2.5 (the bottom of the river is located 2.5m under the topography)

In [10]:
top = disu.top.array
for data in riv.stress_period_data.array[0]:
    node = data[0][0]
    data[1] = top[node] - 0.5
    data[2] = 30.
    data[3] = top[node] - 2.5
    enum_name = data[4]
    data[4] = river_names[enum_name]

print(riv.stress_period_data)

{internal}
([((1396,), 2615.56543162, 30., 2613.56543162, 'CD_LAS_LARA')
 ((1410,), 2603.10831491, 30., 2601.10831491, 'CD_LAS_LARA')
 ((1425,), 2598.00491308, 30., 2596.00491308, 'CD_LAS_LARA') ...
 ((10620,), 1918.98375071, 30., 1916.98375071, 'Q._LOS_CONFINES')
 ((347908,), 1916.61170247, 30., 1914.61170247, 'Q._LOS_CONFINES')
 ((347923,), 1920.30177343, 30., 1918.30177343, 'Q._LOS_CONFINES')])



## Wel package
This package contains all data about wells: wells nodes,  and ther names
This package contains all corresponding nodes to wells. These data are organized as (node, volumetric well rate, name).

In [11]:
wel = model.get_package('wel')
print(wel.stress_period_data)

{internal}
([((24155,), 0., 'well_0') ((24156,), 0., 'well_0')
 ((24157,), 0., 'well_0') ((24158,), 0., 'well_0')
 ((24159,), 0., 'well_0') ((24160,), 0., 'well_0')
 ((24161,), 0., 'well_0') ((24162,), 0., 'well_0')
 ((24163,), 0., 'well_0') ((24164,), 0., 'well_0')
 ((24165,), 0., 'well_0') ((24166,), 0., 'well_0')
 ((24167,), 0., 'well_0') ((24168,), 0., 'well_0')
 ((24169,), 0., 'well_0') ((24170,), 0., 'well_0')
 ((24171,), 0., 'well_0') ((24172,), 0., 'well_0')
 ((24173,), 0., 'well_0') ((24197,), 0., 'well_1')
 ((24198,), 0., 'well_1') ((24199,), 0., 'well_1')
 ((24200,), 0., 'well_1') ((24201,), 0., 'well_1')
 ((24202,), 0., 'well_1') ((24203,), 0., 'well_1')
 ((24204,), 0., 'well_1') ((24205,), 0., 'well_1')
 ((24206,), 0., 'well_1') ((24207,), 0., 'well_1')
 ((24208,), 0., 'well_1') ((24209,), 0., 'well_1')
 ((24210,), 0., 'well_1') ((24211,), 0., 'well_1')
 ((24212,), 0., 'well_1') ((24213,), 0., 'well_1')
 ((24214,), 0., 'well_1') ((24215,), 0., 'well_1')
 ((24238,), 0., 'we

Like rivers data, Geomodelr exports the volumetric well rate for each well equal to 0.0, and the names of the rivers are enumerate from "well_0" to "well_(m-1)", where _m_ is the number of wells in the model. Then, we are going to create a dictyonary using the original name of the wells and their corresponding volumetric rates. Furthemore, we load the wells-name.json file to get the enumerated well name and its corresponding name.

In [12]:
json_file = sim_location + 'wells_name.json'
with open(json_file) as json_file:  
    well_names = json.load(json_file)
    
print("Wells names:\n{}\n".format(well_names))

wells_rate = {u'chinavita_1': -12., u'umbita_1': -13., u'tibana_1': -14., u'chinavita_2': -15.,
		  u'umbita_2': -16., u'tibana_2': -17., u'tibana_3': -18.}

print('Volumetric well rates:\n{}'.format(wells_rate))

Wells names:
{'well_3': 'chinavita_1', 'well_2': 'umbita_2', 'well_1': 'umbita_1', 'well_0': 'tibana_3', 'well_6': 'chinavita_2', 'well_5': 'tibana_2', 'well_4': 'tibana_1'}

Volumetric well rates:
{'chinavita_1': -12.0, 'umbita_1': -13.0, 'tibana_1': -14.0, 'chinavita_2': -15.0, 'umbita_2': -16.0, 'tibana_2': -17.0, 'tibana_3': -18.0}


In [13]:
for data in wel.stress_period_data.array[0]:
    node = data[0][0]
    enum_name = data[2]
    real_name = well_names[enum_name]
    data[1] = wells_rate[real_name]
    data[2] = real_name
print(wel.stress_period_data)

{internal}
([((24155,), -18., 'tibana_3') ((24156,), -18., 'tibana_3')
 ((24157,), -18., 'tibana_3') ((24158,), -18., 'tibana_3')
 ((24159,), -18., 'tibana_3') ((24160,), -18., 'tibana_3')
 ((24161,), -18., 'tibana_3') ((24162,), -18., 'tibana_3')
 ((24163,), -18., 'tibana_3') ((24164,), -18., 'tibana_3')
 ((24165,), -18., 'tibana_3') ((24166,), -18., 'tibana_3')
 ((24167,), -18., 'tibana_3') ((24168,), -18., 'tibana_3')
 ((24169,), -18., 'tibana_3') ((24170,), -18., 'tibana_3')
 ((24171,), -18., 'tibana_3') ((24172,), -18., 'tibana_3')
 ((24173,), -18., 'tibana_3') ((24197,), -13., 'umbita_1')
 ((24198,), -13., 'umbita_1') ((24199,), -13., 'umbita_1')
 ((24200,), -13., 'umbita_1') ((24201,), -13., 'umbita_1')
 ((24202,), -13., 'umbita_1') ((24203,), -13., 'umbita_1')
 ((24204,), -13., 'umbita_1') ((24205,), -13., 'umbita_1')
 ((24206,), -13., 'umbita_1') ((24207,), -13., 'umbita_1')
 ((24208,), -13., 'umbita_1') ((24209,), -13., 'umbita_1')
 ((24210,), -13., 'umbita_1') ((24211,), -13

# IMS package (Iterative Model Solution)
This package contains the information related about numerical solver. This package is registers to simulation and it is assigned to each model.

In [14]:
ims = sim.get_package('ims')
print(ims)

package_name = mf6_webinar
filename = mf6_webinar.ims
package_type = ims
model_or_simulation_package = simulation
simulation_name = modflowsim

Block options
--------------------
print_option
{internal}
(all)

complexity
{internal}
(simple)


Block nonlinear
--------------------
outer_hclose
{internal}
(0.01)

outer_maximum
{internal}
(50)


Block linear
--------------------
inner_maximum
{internal}
(30)

inner_hclose
{internal}
(0.01)

linear_acceleration
{internal}
(cg)

relaxation_factor
{internal}
(0.97)

preconditioner_levels
{internal}
(8)

preconditioner_drop_tolerance
{internal}
(0.0001)





In [15]:
# Maximum number of outer (nonlinear) iterations.
print('Maximum number of nonlinear interations: {}'.format(ims.outer_maximum.get_data()))

# Maximum number of inner (linear) iterations.
print('Maximum number of linear interations: {}'.format(ims.inner_maximum.get_data()))

Maximum number of nonlinear interations: 50
Maximum number of linear interations: 30


In [16]:
# Maximum number of outer (nonlinear) iterations.
ims.outer_maximum.set_data(500)
print('Maximum number of nonlinear interations: {}'.format(ims.outer_maximum.get_data()))

# Maximum number of inner (linear) iterations.
ims.inner_maximum.set_data(130)
print('Maximum number of linear interations: {}'.format(ims.inner_maximum.get_data()))

Maximum number of nonlinear interations: 500
Maximum number of linear interations: 130


In [17]:
# Complexity: Defines if the model can trates as linear or nonlinear problem.
print('Complexity: {}'.format(ims.complexity.get_data()))
# simple: Model can be trated as linear problem (confined units, linear stress packages, etc.)

Complexity: simple


In [18]:
ims.complexity.set_data('moderate')
print('Complexity: {}'.format(ims.complexity.get_data()))
# moderate (several unconfined units, nonlinear stress packages, etc.)

Complexity: moderate


# Newton-Raphson solver
This solver is recommended where traditional wet/drying numerical schemes does not get an acceptable solution due to convergence problems.

In [19]:
# Newton-Rhapson solver using under relaxation 
model.name_file.newtonoptions = ('UNDER_RELAXATION')
# This option allows to under-relax the head where water level falls below bottom of cell.

# Change linear acceleration method 
ims.linear_acceleration.set_data('BICGSTAB')

# write simulation
sim.write_simulation()


writing simulation...
  writing simulation name file...
  writing simulation tdis package...
  writing ims package mf6_webinar...
  writing model mf6_webinar...
    writing model name file...
    writing package disu...
    writing package riv_0...
    writing package wel...
    writing package npf...
    writing package ic...
    writing package oc...


# Run the simulation

In [20]:
#run simulation
sim.run_simulation()

FloPy is using the following  executable to run the model: C:/WRDAPP/mf6.0.4/mf6.0.4/bin/mf6.exe
                                   MODFLOW 6
                U.S. GEOLOGICAL SURVEY MODULAR HYDROLOGIC MODEL
                            VERSION 6.0.4 03/13/2019

   MODFLOW 6 compiled Mar 13 2019 12:37:09 with IFORT compiler (ver. 19.0.0)

This software has been approved for release by the U.S. Geological 
Survey (USGS). Although the software has been subjected to rigorous 
review, the USGS reserves the right to update the software as needed 
pursuant to further analysis and review. No warranty, expressed or 
implied, is made by the USGS or the U.S. Government as to the 
functionality of the software and related material nor shall the 
fact of release constitute any such warranty. Furthermore, the 
software is released on condition that neither the USGS nor the U.S. 
Government shall be held liable for any damages resulting from its 
authorized or unauthorized use. Also refer to the USGS W

(True, [])

# Open results and save it on _vtk_ file
The hydraulic heads are saved in the _.hds_ file. We can get this data using the output_keys command.

In [21]:
keys = sim.simulation_data.mfdata.output_keys()

('mf6_webinar', 'CBC', 'FLOW-JA-FACE')
('mf6_webinar', 'CBC', 'WEL')
('mf6_webinar', 'CBC', 'RIV')
('mf6_webinar', 'HDS', 'HEAD')


In [22]:
# get all head data
head = sim.simulation_data.mfdata[file_name, 'HDS', 'HEAD']
head = head[0]
print(head)

[2547.28251739 2502.62520238 2483.49450224 ... 2247.43731625 2246.88894633
 2246.5390188 ]


It should be noted that when Newton-Raphson method is used no cells will dry. Consequently, we have to mark all cell where head is below its bottom. For this, we create a numpy array called dry_cells.

In [23]:
disu = model.get_package('disu') #First, we get DISU package from the model.
bot = disu.bot.array # Second, we get cells bottom from DISU package.
#mask = head < bot # Mask where val=True if head<bottom and val=False if not.

dry_cells = np.array( head - bot, dtype='float32')

_dry-cells_ is computed as _head - bot_. This array allows us to know which cells can be considerated as _dry_ (if _dry-cells<0_). Now we are ready to load the _vtk_ file and add the head and dry data to visalizate in Paraview.

In [24]:
# Read the vtk source file.
reader = vtk.vtkUnstructuredGridReader()
reader.SetFileName(sim_location + file_name + '.vtk')
reader.Update() # Needed because of GetScalarRange
ugrid = reader.GetOutput()
reader.CloseVTKFile()

# Convert head array to vtk_array and save it on untructured grid (ugrid)
heads = numpy_support.numpy_to_vtk(head.ravel(), deep=True, array_type=vtk.VTK_FLOAT)
heads.SetName('Heads')
ugrid.GetCellData().AddArray(heads)

# Convert dry_cells array to vtk_array and save it on untructured grid (ugrid)
dry_cells = numpy_support.numpy_to_vtk(dry_cells.ravel(), deep=True, array_type=vtk.VTK_FLOAT)
dry_cells.SetName('Dry')
ugrid.GetCellData().AddArray(dry_cells)

8

Finally, we update the _vtk_ file.

In [25]:
writerVTK = vtk.vtkUnstructuredGridWriter()
writerVTK.SetInputData(ugrid)
writerVTK.SetFileName(sim_location + file_name + '.vtk')
writerVTK.Update()

# Reading units nodes
Geomodelr exports a _.json_ file that containts a dictionary where the keys are the geological units names and their values correponds to id nodes of the mesh.

In [26]:
json_file = sim_location + file_name + '.json'
print('JSON file name:\n{}\n'.format(json_file))

with open(json_file) as json_file:  
    unit_nodes = json.load(json_file)

print('Geological units: {}'.format(unit_nodes.keys()))
print('Number of nodes of E2p: {}'.format(len(unit_nodes[u'K2p'])))
print('Number of nodes of E1ss: {}'.format(len(unit_nodes[u'E1ss'])))
print('Number of nodes of E2E3co: {}'.format(len(unit_nodes[u'E2E3co'])))
print('\nFirst 20 nodes of E2E3co: {}'.format(unit_nodes[u'E2E3co'][:10]))

JSON file name:
D:/CompartidaVB/Modflow/Webinar_2019/modflow6/mf6_webinar/mf6_webinar.json

Geological units: dict_keys(['E2p', 'K2p', 'E1ss', 'K2E1g', 'K2lt', 'E1si', 'K1K2U', 'K2cp', 'E2E3co', 'E1c', 'K2d'])
Number of nodes of E2p: 41030
Number of nodes of E1ss: 16730
Number of nodes of E2E3co: 2616

First 20 nodes of E2E3co: [3170, 3171, 3192, 3996, 4015, 4034, 4053, 4072, 4073, 4092]
